In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import sys
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib
import networkx as nx
import pickle
from collections import OrderedDict
import copy
from scipy.sparse import csr_matrix
from scipy import io
import seaborn as sns
import joblib
# from base import *
from joblib import Parallel, delayed
import random
import time

In [3]:
MNM_nb_folder = os.path.join('..', '..','..', '..','..', '..', 'side_project', 'network_builder')
sys.path.append(MNM_nb_folder)
python_lib_folder = os.path.join('..', '..','..', '..','..', 'pylib')
sys.path.append(python_lib_folder)

In [4]:
from MNMAPI import *
from MNM_mcnb import *
from mcDODE import *

/home/lemma/.conda/envs/python2/lib/python2.7/site-packages/bidict/compat.py:33: UserWarning: Python 2 support will be dropped in a future release.
  warn('Python 2 support will be dropped in a future release.')


In [5]:
origin_spc_folder = os.path.join('..' , '..', 'data', 'input_files_MckeesRocks_SPC_PM')
cache_folder = os.path.join('cahce_PM')

In [6]:
nb = MNM_network_builder()

In [7]:
nb.load_from_folder(origin_spc_folder)

In [8]:
nb.config

[DTA]
network_name = Snap_graph
unit_time = 5
total_interval = 4320
assign_frq = 180
start_assign_interval = 0
max_interval = 24
flow_scalar = 2
num_of_link = 16144
num_of_node = 6317
num_of_O = 290
num_of_D = 290
OD_pair = 14084
routing_type = Hybrid
adaptive_ratio = 0.6

[STAT]
rec_mode = LRn
rec_mode_para = 2
rec_folder = record
rec_volume = 0
volume_load_automatic_rec = 0
volume_record_automatic_rec = 0
rec_tt = 1
tt_load_automatic_rec = 0
tt_record_automatic_rec = 0

[FIXED]
path_file_name = path_table
num_path = 14111
buffer_length = 24
choice_portion = Buffer
route_frq = 180

[ADAPTIVE]
route_frq = 180

In [9]:
[m_car, L_car, m_truck, L_truck, 
 m_spd_car, m_spd_truck, observed_link_list] = pickle.load(
                                    open('../generate_observed_data/mc_PM_data.pickle', 'rb'))

In [10]:
data_dict = dict()
data_dict['car_count_agg_L_list'] = [L_car]
data_dict['truck_count_agg_L_list'] = [L_truck]
data_dict['car_link_flow'] = [m_car]
data_dict['truck_link_flow'] = [m_truck]
data_dict['car_link_tt'] = [m_spd_car]
data_dict['truck_link_tt'] = [m_spd_truck]

In [11]:
config = dict()
config['use_car_link_flow'] = True
config['use_car_link_tt'] = False
config['use_truck_link_flow'] = True
config['use_truck_link_tt'] = False
config['car_count_agg'] = True
config['truck_count_agg'] = True
config['link_car_flow_weight'] = 1
config['link_truck_flow_weight'] = 1
config['link_car_tt_weight'] = 0.01
config['link_truck_tt_weight'] = 0.01
config['num_data'] = 1
config['observed_links'] = observed_link_list
config['paths_list'] = range(nb.config.config_dict['FIXED']['num_path'])
config['compute_car_link_flow_loss'] = True
config['compute_truck_link_flow_loss'] = True
config['compute_car_link_tt_loss'] = True
config['compute_truck_link_tt_loss'] = True

In [12]:
dode = MCDODE(nb, config)

In [13]:
dode.add_data(data_dict)

In [ ]:
dode.estimate_path_flow(max_epoch = 100, car_step_size = 0.1, 
                       truck_step_size = 0.001,
                       car_init_scale = 0.01, truck_init_scale = 0.001,
                       adagrad = True, 
                       store_folder = 'pm_mcpath_storage', hashcode = 1,
                       use_file_as_init = "pm_mcpath_storage/third_small_99/96iteration.pickle")

Epoch: 0 Loss: 33401.57 truck_count_loss: 2397.02|car_tt_loss: 341.95|car_count_loss: 30035.84|truck_tt_loss: 626.76|
